https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

The goal is to generate names using the RNN architecture designed in the 1-RNN.

# Imports

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import re
import random
import os

import typing
import string
import unicodedata

# Loading data

In [12]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters) + 1

def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [13]:
categories = []
category_lines = {}

for (dirpath, dirnames, filenames) in os.walk('data/names'):
    for (i, filename) in enumerate([dirpath + os.sep + f for f in filenames]):
        category = os.path.splitext(os.path.basename(filename))[0]
        categories.append(category)
        lines = open(filename, encoding='utf-8').read().lower().strip().split('\n')
        category_lines[category] = [unicode_to_ascii(line) for line in lines]
        
n_categories = len(category_lines)

In [14]:
def input_tensor(name: str) -> torch.Tensor:
    tensor = torch.zeros(len(name), 1, n_letters)
    for (i, c) in enumerate(name):
        tensor[i] = F.one_hot(torch.Tensor([all_letters.find(c)]).long(), num_classes=n_letters)
        
    return tensor

In [22]:
def category_to_tensor(category: str) -> torch.Tensor:
    return F.one_hot(torch.Tensor([categories.index(category)]).long(), num_classes=n_categories)

In [16]:
def target_tensor(name: str) -> torch.Tensor:
    tensor = torch.zeros(len(name), 1, n_letters)
    for (i, c) in enumerate(name):
        if i > 0:
            tensor[i - 1] = F.one_hot(torch.Tensor([all_letters.find(c)]).long(), num_classes=n_letters)
            
    tensor[len(name) - 1] = F.one_hot(torch.Tensor([n_letters - 1]).long(), num_classes=n_letters)
    
    return tensor

# Neural network module

In [17]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_categories):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), dim=1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), dim=1)
        output = self.softmax(self.dropout(self.o2o(output_combined)))
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [18]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_letters, len(categories))

# Training

In [19]:
def get_training_example() -> typing.Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    category = random.choice(categories)
    line = random.choice(category_lines[category])
    category_t = category_to_tensor(category)
    line_t = input_tensor(line)
    target_t = target_tensor(line)
    return category_t, line_t, target_t

In [20]:
criterion = nn.NLLLoss()

n_iters = 100000
learning_rate = 0.0005

def train(category_tensor, input_tensor, target_tensor):
    target_tensor.unsqueeze_(1)
    hidden = rnn.init_hidden()
    
    rnn.zero_grad()
    
    loss = torch.Tensor([0])
    
    for i in range(input_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_tensor[i], hidden)
        l = criterion(output, torch.argmax(target_tensor[i].squeeze(0), dim=1))
        loss += l
        
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return output, loss.item() / input_tensor.size(0)

In [34]:
current_loss = 0
all_losses = []
for iter in range(1, n_iters + 1):
    category_tensor, line_tensor, target_t = get_training_example()
    output, loss = train(category_tensor, line_tensor, target_t)
    all_losses.append(loss)
    if iter % 1000 == 0:
        print(f'Iteration: {iter}, Average loss: {torch.Tensor(all_losses[-100:]).mean()}')
    current_loss += loss

Iteration: 1000, Average loss: 2.3202810287475586
Iteration: 2000, Average loss: 2.334641933441162
Iteration: 3000, Average loss: 2.217543125152588
Iteration: 4000, Average loss: 2.333855152130127
Iteration: 5000, Average loss: 2.3065688610076904
Iteration: 6000, Average loss: 2.3336496353149414
Iteration: 7000, Average loss: 2.3251264095306396
Iteration: 8000, Average loss: 2.295895576477051
Iteration: 9000, Average loss: 2.3418095111846924
Iteration: 10000, Average loss: 2.306250810623169
Iteration: 11000, Average loss: 2.3119490146636963
Iteration: 12000, Average loss: 2.251181125640869
Iteration: 13000, Average loss: 2.3523809909820557
Iteration: 14000, Average loss: 2.303596019744873
Iteration: 15000, Average loss: 2.27984881401062
Iteration: 16000, Average loss: 2.385922431945801
Iteration: 17000, Average loss: 2.279186248779297
Iteration: 18000, Average loss: 2.2081010341644287
Iteration: 19000, Average loss: 2.2171361446380615
Iteration: 20000, Average loss: 2.158221483230591
I

# Evaluation

In [41]:
def generate(category, start_letter, max_length=20):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = category_to_tensor(category)
        input = input_tensor(start_letter)
        hidden = rnn.init_hidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi.item()]
                output_name += letter
            input = input_tensor(letter)

        return output_name

# Now we can generate a name for a given category
for start_letter in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'W', 'Z']:
    print(generate('Arabic', start_letter))

Aasar
Basar
Casar
Dosha
Eosha
Fasan
Gasar
Hasan
Iasar
Jasar
Kasar
Lasar
Masar
Nosha
Oasar
Pasar
Rasar
Sasan
Tasar
Uasan
Wasar
Zasar
